In [1]:
!pip install -q datasets scikit-learn python-crfsuite spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import datasets
from sklearn.model_selection import train_test_split
import pycrfsuite
from sklearn.metrics import classification_report
import json
import re

import spacy

In [2]:
dataset = datasets.load_dataset("adsabs/WIESP2022-NER")
train_data = dataset['train']
dk_data = dataset['validation']
test_data = dataset['test']

nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1753 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1366 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2505 [00:00<?, ? examples/s]

# utils

In [3]:
def count_subtokens(token, subtoken_list):
    count = 0
    for subtoken in subtoken_list:
        if subtoken in token:
            count += 1
    return count

def add_spacy_ner(sentence):
    doc = nlp(" ".join(token for token, _ in sentence))
    spacy_features = [{} for _ in sentence]  # Initialize empty feature dict for each token

    # Match spaCy tokens to the original tokens and assign NER tags
    spacy_index = 0
    for i, (token, _) in enumerate(sentence):
        while spacy_index < len(doc) and doc[spacy_index].idx < len(" ".join(sentence[i][0] for i in range(0, i + 1))):
            if doc[spacy_index].ent_type_:
                spacy_features[i]['spacy_ner_' + doc[spacy_index].ent_type_.lower()] = 1
            spacy_index += 1

    return spacy_features

def search_regex(word, pattern):
    return bool(re.search(pattern, word))


In [4]:
from collections import Counter

entity_name = [
    'Organization',
    'Observatory',
    'CelestialObject',
    'Event',
    'CelestialRegion',
    'Identifier'
]

ner_tags = ["B-"+entity for entity in entity_name] + ["I-"+entity for entity in entity_name]

tag_to_id = {"O" : len(ner_tags)}
for i in range(len(ner_tags)):
    tag_to_id[ner_tags[i]] = i

def process_entity_tag(data, ner_tags=ner_tags):
    """
    Process ner tags based on selected entities

    Args:
        data: (hugging face dateset).
        ner_tags (list): ner tags.

    Returns:
        processed_tags (List[List[str]]): list of processed ner tags, each element is a list of ner tags of a document
        ner_tokens (dic): key is ner tag, value is a list containing all tokens labeled as the tag
        text (str): original text
    """
    # create new ner tags
    processed_tags = []
    text = []
    ner_tokens = {}
    for n in ner_tags:
        ner_tokens[n] = []

    for n in range(len(data)):
        doc = " ".join(data[n]['tokens'])
        ner_copy = data[n]['ner_tags'].copy()
        for i, t in enumerate(ner_copy):
            # taget ner
            if t in ner_tags:
                ner_tokens[t].append(data[n]['tokens'][i])
            # redundant ner
            elif t != "O":
                ner_copy[i] = "O"

        processed_tags.append(ner_copy)
        text.append(doc)

    return (processed_tags, ner_tokens, text)

def find_frequent_subword(tokens, n_gram, top):
    subwords = []
    for t in tokens:
        if len(t)>=n_gram:
            subwords.extend([t[i:i+n_gram] for i in range(len(t)-n_gram+1)])
    counts = Counter(subwords)
    top_subwords = counts.most_common(top)
    return top_subwords

In [5]:
def preprocess_data(dataset):
    processed_tags, _, __ = process_entity_tag(data=dataset)
    formatted_data = []
    i = 0
    for item in dataset:
        tokens = item['tokens']
        sentence = list(zip(tokens, processed_tags[i]))
        formatted_data.append(sentence)
        i += 1
    return formatted_data

def word2features(sent, i, spacy_features, dk=None):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    features.update(spacy_features[i])

    # Domain Knowledge features
    if dk:
        for k, v in dk['sub_tokens'].items():
            features[k] = count_subtokens(word, v)
        for k, v in dk['regex'].items():
            features[k] = search_regex(word, v)

    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
        # Domain Knowledge features
        if dk:
            for k, v in dk['sub_tokens'].items():
                features[f"-1:{k}"] = count_subtokens(word1, v)
            for k, v in dk['regex'].items():
                features[k] = search_regex(word, v)
    else:
        features['BOS'] = True

    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
        # Domain Knowledge features
        if dk:
            for k, v in dk['sub_tokens'].items():
                features[f"+1:{k}"] = count_subtokens(word1, v)
            for k, v in dk['regex'].items():
                features[k] = search_regex(word, v)
    else:
        features['EOS'] = True

    return features

def process_sentence(sentence):
    spacy_features = add_spacy_ner(sentence)
    return [word2features(sentence, i, spacy_features) for i in range(len(sentence))], [label for token, label in sentence]

# generate features for dataset
def prepare_data(sentences, dk):
    X = []
    y = []
    for s in sentences:
        spacy_features = add_spacy_ner(s)
        X.append([word2features(s, i, spacy_features, dk) for i in range(len(s))])
        y.append([label for token, label in s])
    return X, y

# Biased

## Data Preprocess

In [6]:
with open('wholistic_domain_knowledge.json') as json_file:
    wholistic_dk = json.load(json_file)

In [7]:
# Data preprocess
X_test, y_test = prepare_data(preprocess_data(test_data), wholistic_dk)
# split different size


### Testing

In [12]:
import pandas

In [13]:
# Full
tagger = pycrfsuite.Tagger()
tagger.open(r'ner-model-wholistic-full.crfsuite')
y_test_pred = [tagger.tag(xseq) for xseq in X_test]
y_test_flat = [item for sublist in y_test for item in sublist]
y_test_pred_flat = [item for sublist in y_test_pred for item in sublist]

result = classification_report(y_test_flat, y_test_pred_flat, output_dict=True)
df = pandas.DataFrame(result).transpose()
df.to_csv('wholistic-full.csv')
print(result)

{'B-CelestialObject': {'precision': 0.7928307854507116, 'recall': 0.4167359379329454, 'f1-score': 0.5463131129676716, 'support': 3609}, 'B-CelestialRegion': {'precision': 0.3728813559322034, 'recall': 0.10526315789473684, 'f1-score': 0.16417910447761191, 'support': 209}, 'B-Event': {'precision': 0.7894736842105263, 'recall': 0.2542372881355932, 'f1-score': 0.3846153846153846, 'support': 59}, 'B-Identifier': {'precision': 0.9117647058823529, 'recall': 0.34444444444444444, 'f1-score': 0.5, 'support': 180}, 'B-Observatory': {'precision': 0.8907563025210085, 'recall': 0.7194570135746606, 'f1-score': 0.7959949937421776, 'support': 1326}, 'B-Organization': {'precision': 0.9131223543640395, 'recall': 0.794806561979121, 'f1-score': 0.8498663289714367, 'support': 11399}, 'I-CelestialObject': {'precision': 0.7770800627943485, 'recall': 0.4672015101462954, 'f1-score': 0.583554376657825, 'support': 2119}, 'I-CelestialRegion': {'precision': 0.3790322580645161, 'recall': 0.11779448621553884, 'f1-sco

In [14]:
# Half
tagger = pycrfsuite.Tagger()
tagger.open(r'ner-model-wholistic-half.crfsuite')
y_test_pred = [tagger.tag(xseq) for xseq in X_test]
y_test_flat = [item for sublist in y_test for item in sublist]
y_test_pred_flat = [item for sublist in y_test_pred for item in sublist]

result = classification_report(y_test_flat, y_test_pred_flat, output_dict=True)
df = pandas.DataFrame(result).transpose()
df.to_csv('wholistic-half.csv')
print(result)

{'B-CelestialObject': {'precision': 0.7594444444444445, 'recall': 0.37877528401219174, 'f1-score': 0.5054538731743391, 'support': 3609}, 'B-CelestialRegion': {'precision': 0.3050847457627119, 'recall': 0.0861244019138756, 'f1-score': 0.13432835820895522, 'support': 209}, 'B-Event': {'precision': 0.7647058823529411, 'recall': 0.22033898305084745, 'f1-score': 0.3421052631578947, 'support': 59}, 'B-Identifier': {'precision': 0.8767123287671232, 'recall': 0.35555555555555557, 'f1-score': 0.5059288537549407, 'support': 180}, 'B-Observatory': {'precision': 0.8921859545004945, 'recall': 0.6802413273001509, 'f1-score': 0.7719298245614036, 'support': 1326}, 'B-Organization': {'precision': 0.9023752223501099, 'recall': 0.7565575927712958, 'f1-score': 0.8230578354647835, 'support': 11399}, 'I-CelestialObject': {'precision': 0.7912966252220248, 'recall': 0.4204813591316659, 'f1-score': 0.5491525423728814, 'support': 2119}, 'I-CelestialRegion': {'precision': 0.46835443037974683, 'recall': 0.1854636

In [15]:
# 25%
tagger = pycrfsuite.Tagger()
tagger.open(r'ner-model-wholistic-25.crfsuite')
y_test_pred = [tagger.tag(xseq) for xseq in X_test]
y_test_flat = [item for sublist in y_test for item in sublist]
y_test_pred_flat = [item for sublist in y_test_pred for item in sublist]

result = classification_report(y_test_flat, y_test_pred_flat, output_dict=True)
df = pandas.DataFrame(result).transpose()
df.to_csv('wholistic-25.csv')
print(result)

{'B-CelestialObject': {'precision': 0.6953447259870359, 'recall': 0.32696037683568857, 'f1-score': 0.44477949491142105, 'support': 3609}, 'B-CelestialRegion': {'precision': 0.25, 'recall': 0.03827751196172249, 'f1-score': 0.06639004149377593, 'support': 209}, 'B-Event': {'precision': 0.8666666666666667, 'recall': 0.22033898305084745, 'f1-score': 0.3513513513513513, 'support': 59}, 'B-Identifier': {'precision': 0.7936507936507936, 'recall': 0.2777777777777778, 'f1-score': 0.411522633744856, 'support': 180}, 'B-Observatory': {'precision': 0.8409785932721713, 'recall': 0.6221719457013575, 'f1-score': 0.7152145643693109, 'support': 1326}, 'B-Organization': {'precision': 0.894630192502533, 'recall': 0.6971664181068514, 'f1-score': 0.7836505275613845, 'support': 11399}, 'I-CelestialObject': {'precision': 0.7862595419847328, 'recall': 0.291647003303445, 'f1-score': 0.42547332185886405, 'support': 2119}, 'I-CelestialRegion': {'precision': 0.22857142857142856, 'recall': 0.020050125313283207, 'f

In [16]:
# 10%
tagger = pycrfsuite.Tagger()
tagger.open(r'ner-model-wholistic-10.crfsuite')
y_test_pred = [tagger.tag(xseq) for xseq in X_test]
y_test_flat = [item for sublist in y_test for item in sublist]
y_test_pred_flat = [item for sublist in y_test_pred for item in sublist]

result = classification_report(y_test_flat, y_test_pred_flat, output_dict=True)
df = pandas.DataFrame(result).transpose()
df.to_csv('wholistic-10.csv')
print(result)

{'B-CelestialObject': {'precision': 0.6754325259515571, 'recall': 0.27043502355223054, 'f1-score': 0.3862287297190345, 'support': 3609}, 'B-CelestialRegion': {'precision': 0.125, 'recall': 0.009569377990430622, 'f1-score': 0.017777777777777778, 'support': 209}, 'B-Event': {'precision': 1.0, 'recall': 0.01694915254237288, 'f1-score': 0.03333333333333333, 'support': 59}, 'B-Identifier': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 180}, 'B-Observatory': {'precision': 0.6971428571428572, 'recall': 0.46003016591251883, 'f1-score': 0.554293502953203, 'support': 1326}, 'B-Organization': {'precision': 0.8562392179413456, 'recall': 0.6531274673216949, 'f1-score': 0.7410172190703692, 'support': 11399}, 'I-CelestialObject': {'precision': 0.8021806853582555, 'recall': 0.24303916941953752, 'f1-score': 0.37305324157913805, 'support': 2119}, 'I-CelestialRegion': {'precision': 0.2903225806451613, 'recall': 0.022556390977443608, 'f1-score': 0.04186046511627906, 'support': 399}, 'I-Eve